# 4. Wie  entwickelt  sich  der  sportliche  Erfolg  mit  dem  Alter?  Gilt  das  für  die  Spielstatistiken gleichermaßen wie für die Platzierung auf der Rangliste?

In [21]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [22]:
matches = pd.read_csv('../../data/raw/atp_matches_till_2022.csv')
players = pd.read_csv('../../data/raw/atp_players_till_2022.csv')
rankings = pd.read_csv('../../data/raw/atp_rankings_till_2022.csv')

In [23]:
gewinner_anzahl = matches.winner_id.value_counts()
verlierer_anzahl = matches.loser_id.value_counts()

In [24]:
# combine spiele_anzahl and verlierer_anzahl
spiele_anzahl = pd.concat([gewinner_anzahl, verlierer_anzahl], axis=1, sort=False)

In [25]:
spiele_anzahl

,winner_id,loser_id
100284,1275.0,301.0
103819,1265.0,280.0
104745,1078.0,224.0
100656,1075.0,242.0
104925,1045.0,207.0
...,...,...
106728,NaN,1.0
110418,NaN,1.0
210760,NaN,1.0
210764,NaN,1.0


In [26]:
spiele_anzahl['Total'] = spiele_anzahl['winner_id'] + spiele_anzahl['loser_id']

In [27]:
# take rankings and add dob from players (join on player_id in players and player in rankings)
rankings = rankings.merge(players[['player_id', 'dob']], how='left', left_on='player', right_on='player_id')

In [28]:
# convert dob to datetime in format yyyymmdd
rankings['dob'] = pd.to_datetime(rankings['dob'], format='%Y%m%d')
rankings = rankings.rename(columns={'dob': 'birth_date'})
rankings.sort_values(by='player_id')

,ranking_date,rank,player,points,player_id,birth_date
1169016,19770704,366,100001,NaN,100001,1913-11-22
1169419,19780102,414,100001,NaN,100001,1913-11-22
1169888,19780116,397,100001,NaN,100001,1913-11-22
1166893,19750505,399,100002,NaN,100002,1921-06-20
1167847,19751215,373,100002,NaN,100002,1921-06-20
...,...,...,...,...,...,...
1157915,20221219,1470,211758,2.0,211758,NaT
1159907,20221226,1469,211758,2.0,211758,NaT
1158120,20221219,1645,211763,1.0,211763,NaT
1160117,20221226,1650,211763,1.0,211763,NaT


In [29]:
rankings.dropna(inplace=True)

In [30]:
from datetime import datetime

# Convert the date strings to datetime objects
rankings['ranking_date'] = pd.to_datetime(rankings['ranking_date'], format='%Y%m%d')
rankings['birth_date'] = pd.to_datetime(rankings['birth_date'])

# Calculate the age of the players at the time of the ranking
rankings['age'] = rankings['ranking_date'].dt.year - rankings['birth_date'].dt.year
rankings.loc[rankings['ranking_date'].dt.month < rankings['birth_date'].dt.month, 'age'] -= 1
rankings.loc[(rankings['ranking_date'].dt.month == rankings['birth_date'].dt.month) & 
             (rankings['ranking_date'].dt.day < rankings['birth_date'].dt.day), 'age'] -= 1


In [31]:
# drop all rows where age is negative
rankings = rankings[rankings['age'] >= 10]

In [32]:
age_group_stats = rankings.groupby('age').agg({'rank': 'mean', 'points': 'mean'}).reset_index()

# Erstellung des ersten Linienplots für den Durchschnittsrang
fig = go.Figure()
fig.add_trace(go.Scatter(x=age_group_stats['age'], y=age_group_stats['rank'], mode='lines', name='Durchschnittlicher Rang'))

# Erstellung des zweiten Linienplots für die Durchschnittspunkte
#fig.add_trace(go.Scatter(x=age_group_stats['age'], y=age_group_stats['points'], mode='lines', name='Durchschnittliche Punkte', yaxis="y2"))

# Hinzufügen von Achsentiteln und Titel des Diagramms
fig.update_layout(
    title='Durchschnittlicher Rang und Punkte nach Alter',
    xaxis_title='Alter',
    yaxis_title='Durchschnittlicher Rang',
    yaxis2=dict(
        title='Durchschnittliche Punkte',
        overlaying='y',
        side='right'
    )
)

# Anzeigen des Diagramms
fig.show()

# Korrelationsanalyse

In [33]:
correlation_rank_age = rankings['age'].corr(rankings['rank'])
correlation_points_age = rankings['age'].corr(rankings['points'])


In [34]:
print(f"Es gab eine negative Korrelation von {correlation_rank_age.round(4)} zwischen dem Alter und dem Rang, was darauf hindeutet, dass jüngere Spieler tendenziell höhere Ranglistenpositionen einnehmen.\n"\
      f"Die Korrelation zwischen Alter und Punkten war positiv {correlation_points_age.round(4)}, was darauf hindeutet, dass ältere Spieler tendenziell mehr Punkte haben.")

Es gab eine negative Korrelation von -0.2772 zwischen dem Alter und dem Rang, was darauf hindeutet, dass jüngere Spieler tendenziell höhere Ranglistenpositionen einnehmen.
Die Korrelation zwischen Alter und Punkten war positiv 0.1634, was darauf hindeutet, dass ältere Spieler tendenziell mehr Punkte haben.


# Lineare Regression

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Lineare Regression für Alter vs. Rang
regressor_age_rank = LinearRegression()
regressor_age_rank.fit(rankings[['age']], rankings['rank'])
predicted_rank = regressor_age_rank.predict(rankings[['age']])
r2_rank = r2_score(rankings['rank'], predicted_rank)

# Lineare Regression für Alter vs. Punkte
regressor_age_points = LinearRegression()
regressor_age_points.fit(rankings[['age']], rankings['points'])
predicted_points = regressor_age_points.predict(rankings[['age']])
r2_points = r2_score(rankings['points'], predicted_points)

coef_rank = regressor_age_rank.coef_[0]
coef_points = regressor_age_points.coef_[0]


In [36]:
coef_rank, coef_points

(-34.94326539024795, 16.888308014983515)

In [37]:
print(f"Für Alter vs. Rang: Der Koeffizient betrug {coef_rank}, was bedeutet, dass der Rang im Durchschnitt um {coef_rank.round()} Positionen sinkt, je älter ein Spieler wird. \nDas Bestimmtheitsmaß (R²) war {r2_rank}, was eine moderate Vorhersagekraft anzeigt.\n\n"\
    f"Für Alter vs. Punkte: Der Koeffizient betrug {coef_points}, was bedeutet, dass die Punktzahl im Durchschnitt um etwa {coef_points.round()} Punkte steigt, je älter ein Spieler wird. \nDas Bestimmtheitsmaß (R²) war jedoch mit {r2_points} sehr gering.")

Für Alter vs. Rang: Der Koeffizient betrug -34.94326539024795, was bedeutet, dass der Rang im Durchschnitt um -35.0 Positionen sinkt, je älter ein Spieler wird. 
Das Bestimmtheitsmaß (R²) war 0.07682104769321552, was eine moderate Vorhersagekraft anzeigt.

Für Alter vs. Punkte: Der Koeffizient betrug 16.888308014983515, was bedeutet, dass die Punktzahl im Durchschnitt um etwa 17.0 Punkte steigt, je älter ein Spieler wird. 
Das Bestimmtheitsmaß (R²) war jedoch mit 0.026693887679083983 sehr gering.


# Untersuchung auf stochastische Unabhängigkeit: 
Mithilfe eines Chi-Quadrat-Tests können wir prüfen, ob Alter und Ranglistenposition unabhängig voneinander sind

In [38]:
from scipy.stats import chi2_contingency

# Erstellung der Kontingenztafel
contingency_table = pd.crosstab(rankings['age'], rankings['rank'])

# Durchführung des Chi-Quadrat-Tests
chi2, p, dof, expected = chi2_contingency(contingency_table)

chi2, p

(540802.9784717166, 0.0)

Der Chi-Quadrat-Test ergab einen p-Wert von 0.0. (<0.05) Dies bedeutet, dass wir die Nullhypothese der Unabhängigkeit zwischen Alter und Rangliste ablehnen können. 
Der extrem niedrige p-Wert zeigt, dass es eine sehr signifikante Beziehung zwischen dem Alter der Spieler und ihrer Position in der Rangliste gibt. Dies bestätigt, dass der Zusammenhang, den wir in den vorherigen Analysen beobachtet haben, nicht zufällig ist, sondern systematisch.

## Zeitlicher Verlauf der Siegesquoten pro Spieler

In [39]:
def fillna_0_astype_int64(dataframe:pd.DataFrame, columns:list=["Siege", "Niederlagen"]):

    for column in columns:
        dataframe[column] = dataframe[column].fillna(0).astype("int64")
    
    return dataframe

In [40]:
def siegesquote_funktion(dataframe:pd.DataFrame):

    dataframe["Siegesquote"] = dataframe["Siege"] / (dataframe["Siege"]+ dataframe["Niederlagen"])

    return dataframe

In [49]:
#Spielerdaten zum Sieger hinzufügen
df_matches_players = pd.merge(matches, players, how="left", left_on="winner_id", right_on="player_id")

#Spielerdaten zum Verlierer hinzufügen
df_matches_players = pd.merge(df_matches_players, players, how="left", left_on="loser_id", right_on="player_id")

#Spalten umbennen und doppelte rausschmeißen
df_matches_players.rename(columns={"name_first_x":"name_first_winner","name_last_x":"name_last_winner", "dob_x":"dob_winner", "height_x":"height_winner", "wikidata_id_x":"wikidata_winner", 
                                   "name_first_y":"name_first_loser","name_last_y":"name_last_loser", "dob_y":"dob_loser", "height_y":"height_loser", "wikidata_id_y":"wikidata_loser" }, inplace=True)
df_matches_players.drop(["player_id_x", "hand_x", "ioc_x", "player_id_y", "hand_y", "ioc_y"],axis=1, inplace=True)


In [50]:
#Datumsspalten in Datumsformat ändern
df_matches_players["tourney_date"] = pd.to_datetime(df_matches_players["tourney_date"], format='%Y%m%d')
df_matches_players["dob_winner"] = pd.to_datetime(df_matches_players["dob_winner"], format='%Y%m%d')
df_matches_players["dob_loser"] = pd.to_datetime(df_matches_players["dob_loser"], format='%Y%m%d')

In [51]:
# Dann können Sie den .dt-Zugriff anwenden
df_win = df_matches_players.groupby([df_matches_players["winner_id"],df_matches_players["tourney_date"].dt.year]).size().reset_index(name="Siege")
df_lose = df_matches_players.groupby([df_matches_players["loser_id"],df_matches_players["tourney_date"].dt.year]).size().reset_index(name="Niederlagen")
#Tabllen zusammenfügen
df_merge = pd.merge(df_win, df_lose, how="outer", left_on=["winner_id", "tourney_date"], right_on=["loser_id", "tourney_date"])

#Spalten kombinieren, falls eine Spieler_id nur in einem der beiden DataFrames vorkommt, da bspw. in diesem Jahr noch kein Match gewonnen oder verloren wurde
df_merge['Spieler_id'] = df_merge['winner_id'].combine_first(df_merge['loser_id'])

#NaN-Werte entfernen und Datentyp zu int64 ändern
fillna_0_astype_int64(df_merge, ["Spieler_id", "Siege", "Niederlagen"])

#Spalte umbennennen
df_merge.rename(columns={"tourney_date": "Jahr"}, inplace=True)

#DataFrame sortieren
df_merge.sort_values(by=["Spieler_id", "Jahr"], ascending=[True, True], inplace=True)

#Siegesquote berechnen
siegesquote_funktion(df_merge)

#Vor- und Nachname hinzufügen
df_merge = pd.merge(df_merge, players, how="left", left_on="Spieler_id", right_on="player_id")
df_merge.rename(columns={"name_first":"Vorname", "name_last":"Nachname"}, inplace=True)
df_merge = df_merge[["Spieler_id", "Vorname", "Nachname", "Jahr", "Siege", "Niederlagen", "Siegesquote"]]


In [52]:
#Verlauf der Siegesquoten der Top 6 "besten" Spieler herausfiltern
df_top6 = df_merge[(df_merge.Spieler_id == 103819)|(df_merge.Spieler_id == 100284)|(df_merge.Spieler_id == 104745)|(df_merge.Spieler_id == 100656)|(df_merge.Spieler_id == 104925)|(df_filtered.player == 101414)]

/var/folders/jq/xvfn2vg55lsc1wx2_2mrh3640000gn/T/ipykernel_15152/1031508080.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [53]:
fig = px.line(df_top6, x='Jahr', y='Siegesquote', color='Nachname', title='Siegesquote über die Jahre')

fig.update_layout(xaxis_title='Jahr', yaxis_title='Siegesquote')

fig.show()

Man kann deutlich erkennen, dass jeder der Spieler in seinen Anfangsjahren eher schlechte Siegesquoten verzeichnen konnte. Im Laufe der Jahre verbesserte sich diese und gelangte nach 5-10 Jahren an ihr Maximum. Dieses Level wurde dann mehr oder weniger für viele Jahre gehalten. 

Bei den Spielern Connors und Lendl nahm die Quote dann im weiteren Verlauf immer weiter ab, bis sie ab einem bestimmten Jahr gar keine Turniere mehr bestritten haben. Diesen Punkt kann man als Karriereende der Spieler interpretieren. Der Spieler Connors hat zudem noch eine auffällige Siegesquote und 0%, welche wir noch untersuchen werden.

Beim Spieler Federer ist auffällig, dass sein letztes und seit langer Zeit schlechtestes Turnier im Jahr 2021 stattfand. Auch das kann darauf hindeuten, dass seine Karriere wohl ein Ende gefunden hat.

Die Spieler Djokovic und Nadal scheinen noch "im Rennen" zu sein, da ihre Siegesquoten  auch 2022 immer noch auf einem relativ hohen Level sind und sie ihr erstes Turnier erst Jahre später als Federer gespielt haben.

## Zeitlicher Verlauf der Siegesquoten der "besten" 6 Spieler
Da die in unserem Beispiel die "besten" 6 Spieler extrem viele Matches gespielt haben und fast ihr gesamtes Leben nach dem Sport ausrichten, sind diese Spieler repräsentativ dafür, wie sich der sportliche Erfolg mit dem Alter verhält. 
Der Sport wird also immer auf das absolute Optimum getrieben, wodurch die Auswirkungen des Alters besser zu erkennen sind.

In [ ]:
#Datum in Datumsformat bringen
rankings["ranking_date"] = pd.to_datetime(rankings['ranking_date'], format='%Y%m%d')

#Sortieren nach Spieler ID und Datum 
df_filtered = rankings.sort_values(by=["player", "ranking_date"], ascending=True)

#Vor- und Nachname hinzufügen
df_filtered = pd.merge(df_filtered, players, how="left", left_on="player", right_on="player_id")

#Spalten herausfiltern
df_filtered = df_filtered[["ranking_date", "rank", "player", "name_first", "name_last", "points"]]

#Zeilen herausfiltern
df_filtered = df_filtered[(df_filtered.player == 103819)|(df_filtered.player == 100284)|(df_filtered.player == 104745)|(df_filtered.player == 100656)|(df_filtered.player == 104925)|(df_filtered.player == 101414)]

In [ ]:
fig = px.line(df_filtered, x="ranking_date", y='rank', color='name_last', title='Ranglistenplatzierung über die Jahre', labels={"name_last": "Nachname"})

# Layout anpassen (optional)
fig.update_layout(xaxis_title='Jahr', yaxis_title='Rang')

# Diagramm anzeigen
fig.show()